In [1]:
import json 
import numpy as np 
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

In [2]:
data ={"intents": [
    {"tag": "greeting",
     "patterns": ["Hi", "Hey", "Is anyone there?", "Hello", "Hay"],
     "responses": ["Hello", "Hi", "Hi there"]
    },
    {"tag": "goodbye",
     "patterns": ["Bye", "See you later", "Goodbye"],
     "responses": ["See you later", "Have a nice day", "Bye! Come back again"]
    },
    {"tag": "thanks",
     "patterns": ["Thanks", "Thank you", "That's helpful", "Thanks for the help"],
     "responses": ["Happy to help!", "Any time!", "My pleasure", "You're most welcome!"]
    },
    {"tag": "about",
     "patterns": ["Who are you?", "What are you?", "Who you are?" ],
     "responses": ["I.m Joana, your bot assistant", "I'm Joana, an Artificial Intelligent bot"]
    },
    {"tag": "name",
    "patterns": ["what is your name", "what should I call you", "whats your name?"],
    "responses": ["You can call me Joana.", "I'm Joana!", "Just call me as Joana"]
    },
    {"tag": "help",
    "patterns": ["Could you help me?", "give me a hand please", "Can you help?", "What can you do for me?", "I need a support", "I need a help", "support me please"],
    "responses": ["Tell me how can assist you", "Tell me your problem to assist you", "Yes Sure, How can I support you"]
    },
    {"tag": "createaccount",
    "patterns": ["I need to create a new account", "how to open a new account", "I want to create an account", "can you create an account for me", "how to open a new account"],
    "responses": ["You can just easily create a new account from our web site", "Just go to our web site and follow the guidelines to create a new account"]
    },
    {"tag": "complaint",
    "patterns": ["have a complaint", "I want to raise a complaint", "there is a complaint about a service"],
    "responses": ["Please provide us your complaint in order to assist you", "Please mention your complaint, we will reach you and sorry for any inconvenience caused"]
    }
]
}


In [3]:
training_sentences = []
training_labels = []
responses = []
label = []


for intent in data['intents']:
    
    label.append(intent['tag'])
    
    for pattern in intent['patterns']:
        
        training_sentences.append(pattern)
        training_labels.append(intent['tag'])
        responses.append(intent['responses'])
training_sentences             

['Hi',
 'Hey',
 'Is anyone there?',
 'Hello',
 'Hay',
 'Bye',
 'See you later',
 'Goodbye',
 'Thanks',
 'Thank you',
 "That's helpful",
 'Thanks for the help',
 'Who are you?',
 'What are you?',
 'Who you are?',
 'what is your name',
 'what should I call you',
 'whats your name?',
 'Could you help me?',
 'give me a hand please',
 'Can you help?',
 'What can you do for me?',
 'I need a support',
 'I need a help',
 'support me please',
 'I need to create a new account',
 'how to open a new account',
 'I want to create an account',
 'can you create an account for me',
 'how to open a new account',
 'have a complaint',
 'I want to raise a complaint',
 'there is a complaint about a service']

In [4]:
lbl_encoder = LabelEncoder()
lbl_encoder.fit(training_labels)
training_labels = lbl_encoder.transform(training_labels)

training_labels

array([4, 4, 4, 4, 4, 3, 3, 3, 7, 7, 7, 7, 0, 0, 0, 6, 6, 6, 5, 5, 5, 5,
       5, 5, 5, 2, 2, 2, 2, 2, 1, 1, 1])

In [5]:
vocab_size = 1000
embedding_dim = 16
max_len = 20
oov_token = "<OOV>"

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
print(sequences)

[[29], [30], [10, 31, 18], [32], [33], [34], [35, 2, 36], [37], [19], [38, 2], [39, 40], [19, 11, 41, 8], [20, 12, 2], [9, 12, 2], [20, 2, 12], [9, 10, 21, 22], [9, 42, 4, 43, 2], [44, 21, 22], [45, 2, 8, 5], [46, 5, 3, 47, 23], [13, 2, 8], [9, 13, 2, 48, 11, 5], [4, 14, 3, 24], [4, 14, 3, 8], [24, 5, 23], [4, 14, 6, 15, 3, 16, 7], [25, 6, 26, 3, 16, 7], [4, 27, 6, 15, 28, 7], [13, 2, 15, 28, 7, 11, 5], [25, 6, 26, 3, 16, 7], [49, 3, 17], [4, 27, 6, 50, 3, 17], [18, 10, 3, 17, 51, 3, 52]]


In [6]:
padded_sequences = pad_sequences(sequences, truncating='post', maxlen=max_len)
print(padded_sequences)

[[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 29]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 30]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 10 31 18]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 32]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 33]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 34]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 35  2 36]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 37]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 19]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 38  2]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 39 40]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 19 11 41  8]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 20 12  2]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  9 12  2]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 20  2 12]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  9 10

In [7]:
model = Sequential()

model.add(Embedding(vocab_size, embedding_dim, input_length=max_len))
model.add(GlobalAveragePooling1D())
model.add(Dense(16, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(8, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', 
              optimizer='adam', metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 16)            16000     
                                                                 
 global_average_pooling1d (G  (None, 16)               0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 16)                272       
                                                                 
 dense_1 (Dense)             (None, 16)                272       
                                                                 
 dense_2 (Dense)             (None, 8)                 136       
                                                                 
Total params: 16,680
Trainable params: 16,680
Non-trainable params: 0
____________________________________________________

In [8]:
epochs = 500
history = model.fit(padded_sequences, np.array(training_labels), epochs=epochs)

Epoch 1/500
2/2 [==============================] - 1s 18ms/step - loss: 2.0804 - accuracy: 0.0909
Epoch 2/500
2/2 [==============================] - 0s 6ms/step - loss: 2.0790 - accuracy: 0.1212
Epoch 3/500
2/2 [==============================] - 0s 6ms/step - loss: 2.0787 - accuracy: 0.1515
Epoch 4/500
2/2 [==============================] - 0s 6ms/step - loss: 2.0781 - accuracy: 0.1515
Epoch 5/500
2/2 [==============================] - 0s 7ms/step - loss: 2.0774 - accuracy: 0.1515
Epoch 6/500
2/2 [==============================] - 0s 7ms/step - loss: 2.0765 - accuracy: 0.1515
Epoch 7/500
2/2 [==============================] - 0s 7ms/step - loss: 2.0757 - accuracy: 0.1515
Epoch 8/500
2/2 [==============================] - 0s 7ms/step - loss: 2.0749 - accuracy: 0.1515
Epoch 9/500
2/2 [==============================] - 0s 7ms/step - loss: 2.0741 - accuracy: 0.1515
Epoch 10/500
2/2 [==============================] - 0s 5ms/step - loss: 2.0733 - accuracy: 0.1515
Epoch 11/500
2/2 [==========

2/2 [==============================] - 0s 4ms/step - loss: 1.9982 - accuracy: 0.2424
Epoch 85/500
2/2 [==============================] - 0s 2ms/step - loss: 1.9959 - accuracy: 0.2727
Epoch 86/500
2/2 [==============================] - 0s 16ms/step - loss: 1.9940 - accuracy: 0.3030
Epoch 87/500
2/2 [==============================] - 0s 2ms/step - loss: 1.9926 - accuracy: 0.3030
Epoch 88/500
2/2 [==============================] - 0s 0s/step - loss: 1.9909 - accuracy: 0.3030
Epoch 89/500
2/2 [==============================] - 0s 6ms/step - loss: 1.9890 - accuracy: 0.3030
Epoch 90/500
2/2 [==============================] - 0s 690us/step - loss: 1.9860 - accuracy: 0.3030
Epoch 91/500
2/2 [==============================] - 0s 5ms/step - loss: 1.9821 - accuracy: 0.3030
Epoch 92/500
2/2 [==============================] - 0s 9ms/step - loss: 1.9783 - accuracy: 0.3030
Epoch 93/500
2/2 [==============================] - 0s 17ms/step - loss: 1.9753 - accuracy: 0.3030
Epoch 94/500
2/2 [============

2/2 [==============================] - 0s 5ms/step - loss: 1.6884 - accuracy: 0.3636
Epoch 168/500
2/2 [==============================] - 0s 4ms/step - loss: 1.6841 - accuracy: 0.3939
Epoch 169/500
2/2 [==============================] - 0s 419us/step - loss: 1.6800 - accuracy: 0.3939
Epoch 170/500
2/2 [==============================] - 0s 0s/step - loss: 1.6759 - accuracy: 0.4242
Epoch 171/500
2/2 [==============================] - 0s 6ms/step - loss: 1.6715 - accuracy: 0.4242
Epoch 172/500
2/2 [==============================] - 0s 6ms/step - loss: 1.6666 - accuracy: 0.4242
Epoch 173/500
2/2 [==============================] - 0s 8ms/step - loss: 1.6622 - accuracy: 0.4242
Epoch 174/500
2/2 [==============================] - 0s 5ms/step - loss: 1.6579 - accuracy: 0.4242
Epoch 175/500
2/2 [==============================] - 0s 5ms/step - loss: 1.6539 - accuracy: 0.4242
Epoch 176/500
2/2 [==============================] - 0s 5ms/step - loss: 1.6499 - accuracy: 0.4242
Epoch 177/500
2/2 [====

2/2 [==============================] - 0s 4ms/step - loss: 1.3448 - accuracy: 0.4848
Epoch 250/500
2/2 [==============================] - 0s 5ms/step - loss: 1.3393 - accuracy: 0.4848
Epoch 251/500
2/2 [==============================] - 0s 7ms/step - loss: 1.3341 - accuracy: 0.4848
Epoch 252/500
2/2 [==============================] - 0s 5ms/step - loss: 1.3294 - accuracy: 0.4848
Epoch 253/500
2/2 [==============================] - 0s 710us/step - loss: 1.3250 - accuracy: 0.4848
Epoch 254/500
2/2 [==============================] - 0s 4ms/step - loss: 1.3211 - accuracy: 0.4848
Epoch 255/500
2/2 [==============================] - 0s 5ms/step - loss: 1.3174 - accuracy: 0.4848
Epoch 256/500
2/2 [==============================] - 0s 5ms/step - loss: 1.3128 - accuracy: 0.4848
Epoch 257/500
2/2 [==============================] - 0s 6ms/step - loss: 1.3090 - accuracy: 0.4848
Epoch 258/500
2/2 [==============================] - 0s 4ms/step - loss: 1.3060 - accuracy: 0.4848
Epoch 259/500
2/2 [===

2/2 [==============================] - 0s 4ms/step - loss: 1.0807 - accuracy: 0.6970
Epoch 332/500
2/2 [==============================] - 0s 5ms/step - loss: 1.0795 - accuracy: 0.6970
Epoch 333/500
2/2 [==============================] - 0s 3ms/step - loss: 1.0771 - accuracy: 0.6970
Epoch 334/500
2/2 [==============================] - 0s 5ms/step - loss: 1.0736 - accuracy: 0.6970
Epoch 335/500
2/2 [==============================] - 0s 4ms/step - loss: 1.0706 - accuracy: 0.6970
Epoch 336/500
2/2 [==============================] - 0s 2ms/step - loss: 1.0664 - accuracy: 0.6970
Epoch 337/500
2/2 [==============================] - 0s 3ms/step - loss: 1.0612 - accuracy: 0.6970
Epoch 338/500
2/2 [==============================] - 0s 6ms/step - loss: 1.0555 - accuracy: 0.6970
Epoch 339/500
2/2 [==============================] - 0s 1ms/step - loss: 1.0505 - accuracy: 0.6970
Epoch 340/500
2/2 [==============================] - 0s 5ms/step - loss: 1.0471 - accuracy: 0.6970
Epoch 341/500
2/2 [=====

2/2 [==============================] - 0s 11ms/step - loss: 0.8456 - accuracy: 0.7273
Epoch 414/500
2/2 [==============================] - 0s 8ms/step - loss: 0.8425 - accuracy: 0.7576
Epoch 415/500
2/2 [==============================] - 0s 3ms/step - loss: 0.8406 - accuracy: 0.7576
Epoch 416/500
2/2 [==============================] - 0s 12ms/step - loss: 0.8383 - accuracy: 0.7576
Epoch 417/500
2/2 [==============================] - 0s 9ms/step - loss: 0.8349 - accuracy: 0.7576
Epoch 418/500
2/2 [==============================] - 0s 4ms/step - loss: 0.8315 - accuracy: 0.7879
Epoch 419/500
2/2 [==============================] - 0s 5ms/step - loss: 0.8293 - accuracy: 0.8182
Epoch 420/500
2/2 [==============================] - 0s 2ms/step - loss: 0.8273 - accuracy: 0.8182
Epoch 421/500
2/2 [==============================] - 0s 508us/step - loss: 0.8251 - accuracy: 0.8182
Epoch 422/500
2/2 [==============================] - 0s 6ms/step - loss: 0.8223 - accuracy: 0.8182
Epoch 423/500
2/2 [=

2/2 [==============================] - 0s 5ms/step - loss: 0.6334 - accuracy: 0.9091
Epoch 496/500
2/2 [==============================] - 0s 5ms/step - loss: 0.6318 - accuracy: 0.9091
Epoch 497/500
2/2 [==============================] - 0s 4ms/step - loss: 0.6278 - accuracy: 0.9091
Epoch 498/500
2/2 [==============================] - 0s 3ms/step - loss: 0.6225 - accuracy: 0.9091
Epoch 499/500
2/2 [==============================] - 0s 1ms/step - loss: 0.6181 - accuracy: 0.9091
Epoch 500/500
2/2 [==============================] - 0s 7ms/step - loss: 0.6162 - accuracy: 0.9091


In [9]:
model.save("chat_model")

import pickle

with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
with open('label_encoder.pickle', 'wb') as ecn_file:
    pickle.dump(lbl_encoder, ecn_file, protocol=pickle.HIGHEST_PROTOCOL)

INFO:tensorflow:Assets written to: chat_model\assets


In [10]:

import json 
import numpy as np
from tensorflow import keras
from sklearn.preprocessing import LabelEncoder

import colorama 
colorama.init()
from colorama import Fore, Style, Back

import random
import pickle


def chat():
    # load trained model
    model = keras.models.load_model('chat_model')

    # load tokenizer object
    with open('tokenizer.pickle', 'rb') as handle:
        tokenizer = pickle.load(handle)

    # load label encoder object
    with open('label_encoder.pickle', 'rb') as enc:
        lbl_encoder = pickle.load(enc)

    # parameters
    max_len = 20
    
    while True:
        print(Fore.LIGHTBLUE_EX + "User: " + Style.RESET_ALL, end="")
        inp = input()
        if inp.lower() == "quit":
            break

        result = model.predict(keras.preprocessing.sequence.pad_sequences(tokenizer.texts_to_sequences([inp]),
                                             truncating='post', maxlen=max_len))
        tag = lbl_encoder.inverse_transform([np.argmax(result)])

        for i in data['intents']:
            if i['tag'] == tag:
                print(Fore.GREEN + "ChatBot:" + Style.RESET_ALL , np.random.choice(i['responses']))

        # print(Fore.GREEN + "ChatBot:" + Style.RESET_ALL,random.choice(responses))

print(Fore.YELLOW + "Start messaging with the bot (type quit to stop)!" + Style.RESET_ALL)
chat()

Start messaging with the bot (type quit to stop)!
User: quit
